In [80]:
import pandas as pd
import numpy as np
import joblib
import regex as re
import whois
import datetime
import warnings
warnings.filterwarnings('ignore')

# All Functions

In [70]:
def master(url) :
    data = []
    domain, directories, file, parameters = split_url(url, data)
    tld_in_param(parameters, data)
    char(data, url, directories, file, parameters)
    email_in_url(url, data)
    create_exp(domain, data)
    return (np.array(data)).reshape(1, 77)

In [71]:
def split_url(url, data):
    protocol, _, rest = url.partition('://')
    domain, _, rest = rest.partition('/')
    path, _, parameters = rest.partition('?')
    
    directories = path.split('/')
    file = '' if '.' not in directories[-1] else directories[-1]
    directories = directories[:-1] if file else directories
    directories = '/'.join(directories)
    
    parameters = [param.split('=')[1] for param in parameters.split('&')] if parameters else []
    data.append(len(parameters))
    parameters = '/'.join(parameters)
    
    return domain, directories, file, parameters

In [72]:
def tld_in_param(parameters, data) :
    tld = joblib.load('top_level_domains.joblib')
    for i in tld :
        if i in url :
            data.append(1)
            return
    data.append(0)
    return

In [73]:
def char(data, url, directories, file, parameters) :
    components = [url, directories, file, parameters]

    special_characters = '.-_/?=@&! ~,+*#$%'

    for component in components:
        char_count = len(component)
        data.append(char_count)

        for char in special_characters:
            char_count = component.count(char)
            data.append(char_count)
            
    return

In [74]:
def email_in_url(url, data) :
    email_pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,7}\b'
    match = re.search(email_pattern, url)
    if match:
        data.append(1)
    else:
        data.append(0)

In [75]:
def create_exp(domain, data):
    try:
        whois_info = whois.whois(domain)
        try :
            data.append((datetime.datetime.today() - whois_info.get('creation_date')).days)
        except :
            data.append((datetime.datetime.today() - whois_info.get('creation_date')[0]).days)
        try :
            data.append((whois_info.get('expiration_date') - datetime.datetime.today()).days)
        except :
            data.append((whois_info.get('expiration_date')[0] - datetime.datetime.today()).days)
    except Exception as e:
        data.append(0)
        data.append(0)

# Testing

In [76]:
pipe = joblib.load('pipeline.joblib')

In [77]:
url = 'https://www.youtube.com'

In [81]:
pred = pipe.predict(master(url))

In [82]:
if pred[0] == 0 :
    print('May not a phishing website')
else :
    print('May be a phishing website')

May not a phishing website
